[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MouseLand/rastermap/blob/main/notebooks/rastermap_largescale.ipynb)

# Rastermap sorting of 34k neurons

We will use a spontaneous activity recording from [Syeda et al, 2023](https://www.biorxiv.org/content/10.1101/2022.11.03.515121v1.abstract). We recorded 34,086 neurons from mouse sensorimotor cortex for 2+ hours using two-photon calcium imaging at a rate of 3.2Hz. FYI to make the download of the dataset faster, we are analyzing only the first half of the recording. During the recording, the mouse was free to run on an air floating ball, and we recorded the mouse face with a camera at a rate of 50Hz and tracked keypoints on the mouse face.

First we will install the required packages, if not already installed. If on google colab, it will require you to click the "RESTART RUNTIME" button because we are updating numpy. Also if you are on google colab, select the GPU runtime if you want to fit the neural network to predict neural activity from behaviors:
**Runtime > Change runtime type > Hardware accelerator = GPU**

In [ ]:
!pip install numpy>=1.24 # (required for google colab)
!pip install rastermap 
!pip install matplotlib

### Load data and import libraries

If not already downloaded, the following cell will automatically download the processed data stored [here](https://osf.io/8xg7n).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# importing rastermap
# (this will be slow the first time since it is compiling the numba functions)
from rastermap import Rastermap, utils
from scipy.stats import zscore

# download spontaneous activity
filename = utils.download_data(data_type="spont2")

dat = np.load(filename)

# spks is neurons by time
# (each timepoint is 313 ms)
spks = dat["spks"]
n_neurons, n_time = spks.shape
print(f"{n_neurons} neurons by {n_time} timepoints")

# zscore activity (each neuron activity trace is then mean 0 and standard-deviation 1)
spks = zscore(spks, axis=1)

# XY position of each neuron in the recording
xpos, ypos = dat["xpos"], dat["ypos"]

# load the processed behavioral data
tcam, tneural = dat["tcam"], dat["tneural"]
run, beh, beh_names = dat["run"], dat["beh"], dat["beh_names"]
print("Behaviors tracked: ")
print(beh_names)

# colors for the behaviors
kp_colors = np.array([[0.55,0.55,0.55], [0.,0.,1],
                      [0.8,0,0], [1.,0.4,0.2],
                      [0,0.6,0.4], [0.2,1,0.5],
                      ])


### Run Rastermap

Let's sort the single neurons with Rastermap, with clustering and upsampling:

In [ ]:
model = Rastermap(n_clusters=100, # number of clusters to compute
                  n_PCs=128, # number of PCs to use
                  locality=0.75, # locality in sorting to find sequences (this is a value from 0-1)
                  time_lag_window=5, # use future timepoints to compute correlation
                  grid_upsample=10, # default value, 10 is good for large recordings
                ).fit(spks)
y = model.embedding # neurons x 1
isort = model.isort

Let's create superneurons from Rastermap -- we sort the data and then sum over neighboring neurons:

In [ ]:
nbin = 200 # number of neurons to bin over 
sn = utils.bin1d(spks[isort], bin_size=nbin, axis=0) # bin over neuron axis

### Visualization

Use the Rastermap sorting to visualize the neural activity:

In [ ]:
# timepoints to visualize
xmin = 0 
xmax = xmin + 1000

# make figure with grid for easy plotting
fig = plt.figure(figsize=(12,6), dpi=200)
grid = plt.GridSpec(9, 20, figure=fig, wspace = 0.05, hspace = 0.3)

# plot running speed
ax = plt.subplot(grid[0, :-1])
ax.plot(run[xmin:xmax], color=kp_colors[0])
ax.set_xlim([0, xmax-xmin])
ax.axis("off")
ax.set_title("running speed", color=kp_colors[0])

# plot superneuron activity
ax = plt.subplot(grid[1:, :-1])
ax.imshow(sn[:, xmin:xmax], cmap="gray_r", vmin=0, vmax=0.8, aspect="auto")
ax.set_xlabel("time")
ax.set_ylabel("superneurons")

ax = plt.subplot(grid[1:, -1])
ax.imshow(np.arange(0, len(sn))[:,np.newaxis], cmap="gist_ncar", aspect="auto")
ax.axis("off")

Color the neurons by their position in the rastermap:

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(ypos, xpos, s=1, c=y, cmap="gist_ncar", alpha=0.25)
plt.xlabel('X position (um)')
plt.ylabel('Y position (um)')
plt.axis("square")

### Neural activity prediction from behavior

We can use the neural network from [Syeda et al, 2023](https://www.biorxiv.org/content/10.1101/2022.11.03.515121v1.abstract) to predict neural activity from behavior. This will work best if you have a GPU (e.g. on google colab). We will first install the code package with the neural network:

In [ ]:
!pip install neuropop

We will predict the principal components of the neural activity (there are too many single neurons to predict):

In [ ]:
from sklearn.decomposition import TruncatedSVD

# this function returns the left singular vectors scaled by the singular values
Vsv = TruncatedSVD(n_components = 128).fit_transform(spks.T)

# compute the other singular vectors
U = spks @ (Vsv / (Vsv**2).sum(axis=0)**0.5)
U /= (U**2).sum(axis=0)**0.5

Now let's create the neural network to fit:

In [ ]:
from neuropop import nn_prediction
import torch

# ideally we have a GPU we can use ("cuda" option)
device = torch.device("cuda")

# declare the model
pred_model = nn_prediction.PredictionNetwork(n_in=beh.shape[-1], n_kp=22, 
                                             n_out=Vsv.shape[-1])
# put model on the GPU
pred_model.to(device);

print(pred_model)

Train the neural network:

In [ ]:
y_pred_all, ve_all, itest = pred_model.train_model(beh, Vsv, tcam, tneural, delay=-1,
                                                        learning_rate=1e-3, n_iter=400,
                                                    device=device, verbose=True)
Vpred_nl = y_pred_all
itest = itest.flatten() # we run using batches of data so we now flatten it

# variance explained per PC
residual = ((Vpred_nl - Vsv[itest])**2).sum(axis=0)
varexp_PC_nl = 1 - residual / (Vsv[itest]**2).sum(axis=0)

Compute the prediction for the superneurons:

In [ ]:
# principal components for superneurons:
U_sn = utils.bin1d(U[isort], bin_size=nbin, axis=0)

# use U_sn to project from prediction of PCs to superneurons
sn_pred = U_sn @ Vpred_nl.T

Visualize the behavioral prediction on test data:

In [ ]:
fig = plt.figure(figsize=(12, 9), dpi=200)
grid = plt.GridSpec(13, 1, figure=fig, wspace = 0.35, hspace = 0.6)

# plot running speed
ax = plt.subplot(grid[0, 0])
ax.plot(run[itest][xmin:xmax], color=kp_colors[0])
ax.set_xlim([0, xmax-xmin])
ax.axis("off")
ax.set_title("running speed", color=kp_colors[0])

# plot superneuron activity
ax = plt.subplot(grid[1:7, 0])
ax.imshow(sn[:, itest][:, xmin:xmax], cmap="gray_r", vmin=0, vmax=0.85, aspect="auto")
ax.set_ylabel("superneurons")
ax.set_xticks([])
ax.set_title("neural activity")

# plot superneuron prediction
ax = plt.subplot(grid[7:, 0])
ax.imshow(sn_pred[:, xmin:xmax], cmap="gray_r", vmin=0, vmax=0.85, aspect="auto")
ax.set_xlabel("time")
ax.set_ylabel("superneurons")
ax.set_title("behavior prediction")

### Settings

You can see all the rastermap settings with `Rastermap?`

In [ ]:
Rastermap?

### Outputs

All the attributes assigned to the Rastermap `model` are listed with `Rastermap.fit?`

In [ ]:
Rastermap.fit?